In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
%%bigquery dup_event_log_df

WITH mob_cust_price_plan_change_ts AS (
  SELECT
    prod_instnc_ts AS dt,
    BAN,
    MSISDN,
    pp_bus_pp_catlg_itm_cd,
    pp_catlg_itm_nm,
    pp_recur_chrg_amt,
    change_in_pp_recur_chrg_amt,
    0 AS marketing_event,
    '' AS portfolio,
    0 AS CAMPAIGN_NUMBER,
    '' AS CONTROL_GROUP,
    0 AS DELIVERED,
    0 AS OPENED,
    0 AS CLICKTHROUGH,
    0 AS UNSUBSCRIBE, 
    0 AS SMS_OPT_OUT,
    0 AS CONVERSION,
    TIMESTAMP(prod_instnc_ts) AS CONVERSION_DT

  FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.mob_cust_price_plan_change_ts`
),

marketing_measurements AS(
  SELECT 
    CAMPAIGN_IN_HOME_DATE AS dt,
    BAN, 
    CAST(SUBSCRIBER_NO  AS STRING) AS MSISDN,
    '' AS pp_bus_pp_catlg_itm_cd,
    '' AS pp_catlg_itm_nm,
    0 AS pp_recur_chrg_amt,
    0 AS change_in_pp_recur_chrg_amt,
    1 AS marketing_event,
    'DUP' AS portfolio,
    CAMPAIGN_NUMBER,
    CONTROL_GROUP,
    DELIVERED,
    OPENED,
    CLICKTHROUGH,
    UNSUBSCRIBE,
    SMS_OPT_OUT,
    CONVERSION,
    CONVERSION_DT
  FROM `wb-ai-accelerator-1-pr-71edf3.measurements_data.epdmadm_dup_multi_new_ref`    
)

SELECT 
  * 
FROM mob_cust_price_plan_change_ts 
WHERE MSISDN != '0'
UNION ALL 
SELECT 
  * 
FROM marketing_measurements

Data is from 2022 to present

To avoid pennies, we will round change_in_pp_Recur_chrg_amt to nearest dollar

In [ ]:
dup_event_log_df.change_in_pp_recur_chrg_amt=np.round(dup_event_log_df.change_in_pp_recur_chrg_amt, decimals=0)

In [ ]:
dup_event_log_df.head()

Get number of changes 

In [ ]:
dup_event_log_df.loc[dup_event_log_df.marketing_event == 0].MSISDN.nunique()

In [ ]:
num_changes_by_msisdn = dup_event_log_df.loc[dup_event_log_df.marketing_event == 0].MSISDN.value_counts()

In [ ]:
num_changes_by_msisdn = pd.DataFrame(num_changes_by_msisdn).reset_index().rename(columns={'MSISDN' : 'count_price_plan_changes', 'index' : 'MSISDN'})

In [ ]:
num_changes_by_msisdn

In [ ]:
num_changes_by_msisdn.loc[num_changes_by_msisdn.count_price_plan_changes==1].shape

Number of changes including original price. So subtracting 1, amongst the people that did change there price plan, they changed on average 1.4 times

In [ ]:
num_changes_by_msisdn.loc[num_changes_by_msisdn.count_price_plan_changes!=1].describe()

Amongst the people that changed their price plan. Lets see the delta

In [ ]:
price_plan_change_delta = dup_event_log_df.loc[(dup_event_log_df.marketing_event == 0) & 
                                               (dup_event_log_df.MSISDN.isin(num_changes_by_msisdn.loc[num_changes_by_msisdn.count_price_plan_changes!=1].MSISDN)) &
                                               (dup_event_log_df.change_in_pp_recur_chrg_amt!=0)].change_in_pp_recur_chrg_amt.value_counts()

In [ ]:
price_plan_change_delta = pd.DataFrame(price_plan_change_delta).reset_index().rename(columns={'index' : 'change_in_pp_recur_chrg_amt', 'change_in_pp_recur_chrg_amt' : 'count'})


In [ ]:
price_plan_change_delta.sort_values(by='change_in_pp_recur_chrg_amt', ascending=True)

In [ ]:
price_plan_change_delta.sort_values(by='count', ascending=False)

Plotting the count vs change in pp recur chrg amt

In [ ]:
fig = px.line(price_plan_change_delta.sort_values(by='change_in_pp_recur_chrg_amt', ascending=True), x='change_in_pp_recur_chrg_amt', y='count')
fig.show()            

In [ ]:
dup_event_log_df.loc[dup_event_log_df.MSISDN.isin(dup_event_log_df.loc[(dup_event_log_df.change_in_pp_recur_chrg_amt <= -50) & (dup_event_log_df.pp_recur_chrg_amt!=0)].MSISDN.unique())].sort_values(by=['BAN', 'dt'])

In [ ]:
dup_event_log_df.loc[dup_event_log_df.MSISDN=='6139144658'].sort_values(by='dt', ascending=True)

In [ ]:
dup_event_log_df.loc[dup_event_log_df.MSISDN=='4032820108'].sort_values(by='dt', ascending=True)

In [ ]:
dup_event_log_df.loc[(dup_event_log_df.change_in_pp_recur_chrg_amt <= -50) & (dup_event_log_df.pp_recur_chrg_amt!=0)]

## Marketing Impact

In [ ]:
dup_event_log_df.loc[(dup_event_log_df.marketing_event==1) & (dup_event_log_df.CONVERSION==1)]

In [ ]:
dup_event_log_df.loc[dup_event_log_df.MSISDN=='4038192963'].sort_values(by='dt', ascending=True)

In [ ]:
dup_event_log_df.loc[(dup_event_log_df.marketing_event==1) & (dup_event_log_df.UNSUBSCRIBE==1)]

In [ ]:
dup_event_log_df.loc[dup_event_log_df.MSISDN=='4182913503'].sort_values(by='dt', ascending=True)

In [ ]:
%%bigquery unsub_dup_df


WITH unsubscribe_msisdn_date AS (
SELECT  
  BAN,
  MSISDN,
  MAX(dt) AS unsub_date
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_dup_event_ts` 
WHERE UNSUBSCRIBE = 1
GROUP BY BAN, MSISDN
)

SELECT 
  A.*  
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_dup_event_ts` A 
INNER JOIN unsubscribe_msisdn_date B 
ON A.BAN = B.BAN AND A.MSISDN = B.MSISDN 
WHERE 
  A.dt >= B.unsub_date
  AND A.marketing_event = 0 -- natural price plan changes only 
  AND A.pp_recur_chrg_amt != 0 -- I believe when pp_recur_chrg_amt becomes 0, it basically means they stopped their plan
ORDER BY BAN, MSISDN, dt 


In [ ]:
unsub_dup_df.change_in_pp_recur_chrg_amt.describe()

In [ ]:
%%bigquery converted_dup_df

WITH unsubscribe_msisdn AS (
SELECT  
  distinct MSISDN as unsub_msisdn,
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.mob_cust_price_plan_change_dup_event_ts` 
WHERE UNSUBSCRIBE = 1

),

customers_with_conversion AS (
  SELECT 
    distinct MSISDN as converted_msisdn
  FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.mob_cust_price_plan_change_dup_event_ts` 
  WHERE CONVERSION = 1 AND MSISDN NOT IN (SELECT unsub_msisdn FROM unsubscribe_msisdn)

)

SELECT
  A.*
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.mob_cust_price_plan_change_dup_event_ts` A 
WHERE
  A.marketing_event = 0 
  AND A.pp_recur_chrg_amt != 0 
  AND A.MSISDN IN (SELECT converted_msisdn FROM customers_with_conversion)

In [ ]:
converted_dup_df.change_in_pp_recur_chrg_amt.describe()

In [ ]:
%%bigquery converted_dup_df_2

WITH unsubscribe_msisdn AS (
SELECT  
  distinct MSISDN as unsub_msisdn,
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_dup_event_ts` 
WHERE UNSUBSCRIBE = 1

),

customers_with_conversion AS (
  SELECT 
    BAN,
    MSISDN,
    MAX(dt) AS convert_date
  FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_dup_event_ts` 
  WHERE CONVERSION = 1 AND MSISDN NOT IN (SELECT unsub_msisdn FROM unsubscribe_msisdn)
  GROUP BY BAN, MSISDN
)

SELECT
  A.*
FROM `wb-ai-accelerator-1-pr-71edf3.mobility_cust_journey.post_mob_cust_price_plan_change_dup_event_ts` A 
INNER JOIN customers_with_conversion B
ON A.BAN = B.BAN AND A.MSISDN = B.MSISDN
WHERE
  A.dt >= B.convert_date AND 
  A.marketing_event = 0 
  AND A.pp_recur_chrg_amt != 0 
  
  

In [ ]:
converted_dup_df_2.change_in_pp_recur_chrg_amt.describe()